<a href="https://colab.research.google.com/github/pmendizabal/crash_course_on_analytics_with_python/blob/main/Pandas_1_Inegi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Práctica 1, Información INEGI

In [ ]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/Data_Analysis_Pandas')

In [ ]:
#data source www.inegi.org.mx/programas/enoe/15ymas/default.html#Datos_abiertos
#documentation www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/fd_c_bas_amp_15ymas.pdf
#municipalities codes: https://www.gob.mx/cms/uploads/attachment/file/196925/Atenci_n_especial_2017.pdf
!pip install geopandas
import pandas as pd, geopandas as gpd
import numpy as np
import matplotlib
import warnings
from copy import deepcopy
warnings.filterwarnings('ignore')

# Recomendaciones:
# 1. usar nombres sin espacios para poder usar después nomenclatura df.nombre_de_columa
# 2. si una columna ya no te sirve, mejor hacerle drop('nombre_col', axis =1, inplace=True)
#     porque entre menos columnas en memoria, más rápidos son los procesos
# 3. es más eficiente importar solo las columnas que vas a necesitar
# 4. usar replace para renombrar valores

In [ ]:
"""
sex: sexo, eda: edad, clase2: ocupado o no, 
cs_p13_1: grado de estudios
pos_ocu: posición en la ocupación
rama: sector de actividad económica
c_ocu11c: tipo de trabajo (oficinista, trabajor de la educación, etx)
ingocup: ingresos mensuales
hrsocup: horas de trabajo a la semana
cs_p17: estudiante o no
rama_est2: sector de actividades (agricultura, etc)
fac: factor de ocupación
ent: estado
mun: municipio
"""
#------------  reading datasets  ----------------
municipalities = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data_Analysis_Pandas/municipalities_codes.csv', index_col=0)

enoe = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data_Analysis_Pandas/demographic_data.csv',encoding='latin',
                   usecols=["sex", "eda", "clase2", "cs_p13_1", "pos_ocu", "rama",
                            "c_ocu11c","ingocup","hrsocup", "cs_p17", "rama_est2", 
                            "fac", "ent", "mun"])
enoe.head()

In [ ]:
#Con esta condicion nos quedamos con los ocupados del trimestre
enoe_ocu = enoe[enoe['clase2'] == 1]
#Con esta condicion nos quedamos con la gente de CDMX
enoe_ocu = enoe_ocu[enoe_ocu['ent'] == 9]

In [ ]:
#Creamos una columna para recodificar la edad
#Aquí solo cambiamos el tipo de dato de la columna
enoe_ocu['eda'] = pd.to_numeric(enoe_ocu['eda'])
#Set condiciones
conditions = [
    (enoe_ocu['eda'] >= 0) & (enoe_ocu['eda'] <= 15),
    (enoe_ocu['eda'] >= 16) & (enoe_ocu['eda'] <= 20),
    (enoe_ocu['eda'] >= 21) & (enoe_ocu['eda'] <= 30),
    (enoe_ocu['eda'] >= 31) & (enoe_ocu['eda'] <= 40),
    (enoe_ocu['eda'] >= 41) & (enoe_ocu['eda'] <= 50),
    (enoe_ocu['eda'] >= 51)
]

choices = ['1', '2', '3', '4', '5', '6']
#Ocupamos la función de select para crear la columna con los objetos predefinidos 
enoe_ocu['rango_edad'] = np.select(conditions, choices)
enoe_ocu

In [ ]:
#Otra forma más limpia es
age_bins = [0,15,20,30,40,50,150]
#La función cut se ocupa para segmentar y ordenar datos dado intervalos (bins)
enoe_ocu['eda'] = pd.to_numeric(enoe_ocu['eda'])
enoe_ocu.loc[:,'eda'] = pd.cut(enoe_ocu.eda, age_bins, labels = [1,2,3,4,5,6],include_lowest=True)
enoe_ocu

In [ ]:
#Ahora para los ingresos, que en este caso solo son salarios
wage_bins = [0,1000,5000,10000,20000,35000,1000000]
enoe_ocu.loc[:,'ingocup'] = pd.cut(enoe_ocu.ingocup.astype('float'),
        wage_bins, labels = ['ultra_low_income', 'low_income', 'medium_income', 
        'regular_income', 'high_income', 'super_high_income'],include_lowest=True)
enoe_ocu

In [ ]:
dict_sex = {1:'Hombre',2:'Mujer'}
#Dado que hay muchas clasificaciones, es mejor crear diccionarios
#El metodo enumerate, fija las posiciones y debemos marcarle que inicie en 1 sino pone 0 por defecto
dict_pos = dict(enumerate(['Trabajadores subordinados', 'Empleadores', 'Cuenta propia'],1))
dict_rama = dict(enumerate(['construccion', 'manufactura', 'comercio', 
           'servicios', 'otros', 'agropecuario', 'no especificado'],1))
dict_cocu11c = dict(enumerate(['Profesionales, tecnicos y trabajadores del arte', 
                'Trabajadores de la educacion','Funcionarios y directivos', 
                'Oficinistas', 'Trabajadores industriales, artesanos y ayudantes', 
                'Comerciantes', 'Operadores de transporte',
                'Trabajadores en servicios personales', 
                'Trabajadores en proteccion y vigilancia'],1))
dict_ramaest2 = dict(enumerate(['Agro', 'Minería y electricidad', 'Manufactura', 
                    'Construcción', 'Comercio', 'Restaurantes y hoteles', 
                    'Transportes, correo y almacenamiento', 
                    'Servicios profesionales, financieros y corporativos', 
                    'Servicios sociales', 'Servicios diversos', 
                    'Gobierno y organismos internacionales'],1))
dict_csp13 = {0:'Ninguno', 1:'Preescolar',2:'Primaria',3:'Secundaria', 
              4:'Prepa o bachiller', 5:'Normal', 6:'Carrera tecnica',
              7:'Profesional',8:'Maestria',9:'Doctorado',99:'No sabe'}
dict_csp17 = {'1':'Estudiante', '2':'No estudiante','9': 'NR'}

In [ ]:
#Reemplazamos dichos valores en las columnas 
enoe_ocu.loc[:,'sex'].replace(dict_sex,inplace = True)
enoe_ocu.loc[:,'pos_ocu'].replace(dict_pos,inplace=True)
enoe_ocu.loc[:,'rama'].replace(dict_rama,inplace=True)
enoe_ocu.loc[:,'c_ocu11c'].replace(dict_cocu11c,inplace=True)
enoe_ocu.loc[:,'rama_est2'].replace(dict_ramaest2,inplace=True)
enoe_ocu.loc[:,'cs_p13_1'].replace(dict_csp13,inplace=True)
enoe_ocu.loc[:,'cs_p17'].replace(dict_csp17,inplace=True)

enoe_ocu

In [ ]:
#Renombramos las columnas
enoe_ocu.rename(columns= {'sex':'Sexo','eda':'Rango edad','cs_p13_1': 'Nivel estudios',
                'pos_ocu': 'Posicion ocupacion','rama':'Rama Eco', 
                'clase2':'Condicion en la ocupacion','ingocup': 'Rango ingresos', 
                'hrsocup':'Horas trabajadas a la semana','cs_p17': 'Estudiante',
                'rama_est2':'Subrama eco','fac' :'Factor','ent':'Estado',
                'mun': 'Municipio','c_ocu11c':'Condicion en la ocupacion'},inplace = True)
enoe_ocu

In [ ]:
#Cambiamos tipos de datos en algunas columnas
enoe_ocu.loc[:,'Rango edad'] = enoe_ocu.loc[:,'Rango edad'].astype('int')
enoe_ocu.loc[:,'Horas trabajadas a la semana'] = enoe_ocu.loc[:,
                                              'Horas trabajadas a la semana'].astype('int')
enoe_ocu.loc[:,'Rango edad'] = enoe_ocu.loc[:,'Rango edad'].astype('int')
#Borramos los diccionarios
del age_bins, wage_bins, dict_sex, dict_pos,dict_rama,dict_cocu11c,dict_ramaest2
del dict_csp13, dict_csp17

In [ ]:
#Ahora hay que vincular con los municipios
enoe_ocu.loc[:,'Municipio'] = enoe_ocu.loc[:,['Estado','Municipio']].apply(
    lambda x: str(int(x[0])) +'-'+ str(int(x[1])), axis=1) #encoding municipalities
print(municipalities.head())
municipalities = pd.Series(municipalities.municipality.values,
                            index=municipalities.code).to_dict()
enoe_ocu.Municipio.replace(municipalities, inplace= True)

In [ ]:
#Analizar mediante clasificaciones de ingreso
regular = enoe_ocu.loc[enoe_ocu['Rango ingresos'] == 'regular_income',:]
high = enoe_ocu.loc[enoe_ocu['Rango ingresos'] == 'high_income',:]
super_high = enoe_ocu.loc[enoe_ocu['Rango ingresos'] == 'super_high_income',:]

#Ahora utilizamos el factor de expansion de la encuesta para tener la estimacion poblacional
regular = pd.DataFrame(np.repeat(regular.values, 
                                 regular['Factor'].values, axis= 0), columns = enoe_ocu.columns)
high = pd.DataFrame(np.repeat(high.values,
                              high['Factor'].values, axis= 0), columns = enoe_ocu.columns)
super_high = pd.DataFrame(np.repeat(super_high.values, super_high['Factor'].values, axis= 0), 
                             columns = enoe_ocu.columns)

In [ ]:
regular.describe()

In [ ]:
regular_cross = pd.crosstab(regular['Estudiante'],regular['Posicion ocupacion'], margins=False)
regular_cross

In [ ]:
regular_cross = pd.crosstab(regular['Rama Eco'],regular['Posicion ocupacion'], margins=False)
regular_cross